In [1]:
# based on https://github.com/theRealSuperMario/supermariopy/blob/master/scripts/tflogs2pandas.py

In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [5]:
def get_metrics(parameters):
    
    DEFAULT_SIZE_GUIDANCE = {
    "compressedHistograms": 1,
    "images": 1,
    "scalars": 0,  # 0 means load all
    "histograms": 1,
    }
    
    # uncomment the following line if models are in local disk
    _ROOT = '/home/developer/gcp'
    # uncomment the following line for execution in Google Storage
    # _ROOT = 'gs:/'
    _STEM = 'cbidmltsf/models'
    _MODEL = parameters['model_stem']
    
    _PATHS = list()
    # how many experiments to collect?
    for row in range(parameters['no_executions']):
        _PATHS.append('{}/{}/{}_{:02d}'.format(_ROOT, _STEM, _MODEL, row))
    
    # create an empty Dataframe with column names only
    metrics_df = pd.DataFrame(columns=['id', 'exp',
                                       'm',
                                       'hourly',
                                       'daily',
                                       'weekly',
                                       'use_timestamps',
                                       'dense',
                                       'train_specs',
                                       'train_metric', 'train_value', 'train_step',
                                       'eval_metric', 'eval_value', 'eval_step'])

    for _PATH in _PATHS:
        
        # it only works if id_exp are in the shape 'TPU_NN_XX'!!!
        id = _PATH[-9:-3]
        exp = int(_PATH[-2:])
        
        r = dict()

        # r['id'] = [id] # use here an iterable to avoid setting up an index
        # iterable is not needed anymore
        r['id'] = id
        r['exp'] = exp

        r['m'] = parameters['m']
        r['hourly'] = parameters['hourly']
        r['daily'] = parameters['daily']
        r['weekly'] = parameters['weekly']
        r['use_timestamps'] = parameters['use_timestamps']
        r['dense'] = parameters['dense']
        r['train_specs'] = parameters['train_specs']

        event_acc = EventAccumulator(_PATH, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]

        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            # r = {"metric": [tag] * len(step), "value": values, "step": step}
            r['train_metric'] = 'train_{}'.format(tag)
            r['train_value'] = values[-1]
            r['train_step'] = step[-1]

        _EVAL_PATH = '{}/eval'.format(_PATH)

        event_acc = EventAccumulator(_EVAL_PATH, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()

        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            # r = {"metric": ['eval_{}'.format(tag)] * len(step), "value": values, "step": step}
            r['eval_metric'] = 'eval_{}'.format(tag)
            r['eval_value'] = values[-1]
            r['eval_step'] = step[-1]

        metrics_df = metrics_df.append(r, ignore_index=True)
        
    return metrics_df


In [6]:
parameters = dict()

In [7]:
_PROCESS_ID = 'TPU_10'
chromosome = [[16, 8, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.005, 16000, 10000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '166404_083202_043201_TPU_10',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [8]:
_PROCESS_ID = 'TPU_11'
chromosome = [[8, 4, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.005, 16000, 10000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_043202_043201_TPU_11',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [9]:
_PROCESS_ID = 'TPU_13'
chromosome = [[8, 4, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_043202_043201_TPU_13',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [10]:
_PROCESS_ID = 'TPU_14'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086402_043201_TPU_14',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [11]:
_PROCESS_ID = 'TPU_15'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 2, 1],
              False,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086402_043201_TPU_15',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [12]:
_PROCESS_ID = 'TPU_16'
chromosome = [[8, 8, 4],
              [64, 32, 32],
              [3, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086403_083202_043201_TPU_16',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [13]:
_PROCESS_ID = 'TPU_17'
chromosome = [[8, 8, 4],
              [64, 32, 32],
              [1, 1, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086401_083201_043201_TPU_17',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [14]:
_PROCESS_ID = 'TPU_18'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_18',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [15]:
_PROCESS_ID = 'TPU_19'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [64, 32, 16, 8],
              [0.004, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_19',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [16]:
_PROCESS_ID = 'TPU_20'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_20',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [17]:
_PROCESS_ID = 'TPU_21'
chromosome = [[8, 8, 4],
              [64, 64, 64],
              [4, 4, 4],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_046404_TPU_21',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [18]:
_PROCESS_ID = 'TPU_22'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 64],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_22',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [19]:
_PROCESS_ID = 'TPU_23'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 256, 64],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_23',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [20]:
_PROCESS_ID = 'TPU_24'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_24',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [21]:
_PROCESS_ID = 'TPU_25'
chromosome = [[8, 8, 4],
              [96, 96, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '089604_089604_043202_TPU_25',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [22]:
_PROCESS_ID = 'TPU_26'
chromosome = [[8, 8, 4],
              [48, 48, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '084804_084804_043202_TPU_26',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [23]:
_PROCESS_ID = 'TPU_27'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_27',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [24]:
_PROCESS_ID = 'TPU_30'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_30',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [25]:
_PROCESS_ID = 'TPU_31'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_31',
    'm': chromosome[0],
    'hourly': [16, 64, 128],    
    'daily': [16, 64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [26]:
_PROCESS_ID = 'TPU_32'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_32',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [27]:
_PROCESS_ID = 'TPU_33'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_33',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [28]:
_PROCESS_ID = 'TPU_34'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [1024, 256, 16],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_34',
    'm': chromosome[0],
    'hourly': [64, 256],    
    'daily': [64, 256],    
    'weekly': [16, 128],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [29]:
_PROCESS_ID = 'TPU_35'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True, # timestamps were used inside the dense layer
              [1024, 256, 16],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_35',
    'm': chromosome[0],
    'hourly': [64, 256],    
    'daily': [64, 256],    
    'weekly': [16, 128],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [30]:
_PROCESS_ID = 'TPU_36'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_36',
    'm': chromosome[0],
    'hourly': [64, 256, 128],    
    'daily': [64, 256, 128],    
    'weekly': [32, 128, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 9
}

In [31]:
_PROCESS_ID = 'TPU_37'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              False,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_37',
    'm': chromosome[0],
    'hourly': [64, 256, 128],    
    'daily': [64, 256, 128],    
    'weekly': [32, 128, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [32]:
_PROCESS_ID = 'TPU_38'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_38',
    'm': chromosome[0],
    'hourly': [64, 128, 256],    
    'daily': [64, 128, 256],    
    'weekly': [32, 64, 128],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [33]:
_PROCESS_ID = 'TPU_39'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_39',
    'm': chromosome[0],
    'hourly': [64, 128, 256, 128],    
    'daily': [64, 128, 256, 128],    
    'weekly': [16, 64, 128, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [34]:
_PROCESS_ID = 'TPU_40'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_40',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [64, 128],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [35]:
_PROCESS_ID = 'TPU_41'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_41',
    'm': chromosome[0],
    'hourly': [32, 128],    
    'daily': [32, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [36]:
_PROCESS_ID = 'TPU_42'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.005, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_42',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [37]:
_PROCESS_ID = 'TPU_50'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.005, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_50',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 30
}

In [38]:
_PROCESS_ID = 'TPU_51'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.001, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_51',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 30
}

In [39]:
_PROCESS_ID = 'TPU_52'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.001, 31000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_52',
    'm': chromosome[0],
    'hourly': [64, 128],    
    'daily': [64, 128],    
    'weekly': [16, 64],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 20
}

In [40]:
metrics = dict()

In [41]:
for key in parameters.keys():
    metrics[key] = get_metrics(parameters[key])

W0324 10:13:03.643772 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:03.650636 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:03.657367 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:03.662695 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:03.728218 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:04.295335 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:04.297488 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:04.346811 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:04.349151 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:04.351870 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:05.000828 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:05.003214 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:05.049435 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:05.052453 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:05.102098 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:06.033908 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:06.036101 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:06.079546 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:06.081921 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:06.127304 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:06.955821 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:06.957715 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:06.988141 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:06.989674 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:07.021131 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

...

W0324 10:13:22.343230 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:22.345364 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:22.391990 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:22.394140 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:22.434465 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:23.290553 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:23.293423 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:23.335405 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:23.337679 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:23.379629 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:24.233691 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:24.235868 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:24.276372 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:24.278347 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:24.320000 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:25.193548 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:25.196203 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:25.251408 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:25.253785 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:25.256269 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

W0324 10:13:25.819632 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:25.822122 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:25.824460 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0324 10:13:25.826692 140320763561728 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0324 10:13:25.876187 140320763561728 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

In [42]:
metrics['TPU_10'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000547,0.000588
std,0.000657,0.000412
min,0.000059,0.000290
25%,0.000214,0.000394
50%,0.000265,0.000481
75%,0.000348,0.000555
max,0.001937,0.001728


In [43]:
metrics['TPU_11'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000407,0.000528
std,0.000285,0.000098
min,0.000160,0.000403
25%,0.000192,0.000452
50%,0.000333,0.000530
75%,0.000511,0.000579
max,0.001060,0.000712


In [44]:
metrics['TPU_13'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.005231,0.002390
std,0.014825,0.005460
min,0.000149,0.000441
25%,0.000344,0.000492
50%,0.000426,0.000652
75%,0.000579,0.000817
max,0.047402,0.017915


In [45]:
metrics['TPU_14'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000524,0.000470
std,0.000531,0.000254
min,0.000076,0.000210
25%,0.000127,0.000271
50%,0.000306,0.000411
75%,0.000879,0.000533
max,0.001460,0.000955


In [46]:
metrics['TPU_15'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000679,0.000854
std,0.000999,0.000424
min,0.000034,0.000443
25%,0.000222,0.000587
50%,0.000325,0.000739
75%,0.000647,0.000932
max,0.003444,0.001891


In [47]:
metrics['TPU_16'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000611,0.000579
std,0.000730,0.000194
min,0.000174,0.000391
25%,0.000208,0.000406
50%,0.000317,0.000551
75%,0.000492,0.000687
max,0.002481,0.000911


In [48]:
metrics['TPU_17'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000692,0.000936
std,0.000395,0.000161
min,0.000213,0.000753
25%,0.000364,0.000815
50%,0.000617,0.000904
75%,0.000992,0.001036
max,0.001395,0.001249


In [49]:
metrics['TPU_18'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000519,0.000401
std,0.000559,0.000111
min,0.000036,0.000210
25%,0.000240,0.000381
50%,0.000394,0.000408
75%,0.000490,0.000430
max,0.002034,0.000621


In [50]:
metrics['TPU_19'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000566,0.000430
std,0.000657,0.000082
min,0.000047,0.000308
25%,0.000189,0.000377
50%,0.000401,0.000439
75%,0.000653,0.000468
max,0.002295,0.000543


In [51]:
metrics['TPU_20'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000247,0.000293
std,0.000262,0.000155
min,0.000041,0.000139
25%,0.000081,0.000198
50%,0.000181,0.000233
75%,0.000295,0.000365
max,0.000928,0.000659


In [52]:
metrics['TPU_21'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000524,0.000448
std,0.000327,0.000175
min,0.000155,0.000185
25%,0.000246,0.000321
50%,0.000440,0.000432
75%,0.000840,0.000582
max,0.000962,0.000728


In [53]:
metrics['TPU_22'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000444,0.000529
std,0.000376,0.000179
min,0.000020,0.000325
25%,0.000155,0.000378
50%,0.000352,0.000499
75%,0.000593,0.000644
max,0.001194,0.000855


In [54]:
metrics['TPU_23'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000380,0.000353
std,0.000368,0.000169
min,0.000039,0.000161
25%,0.000210,0.000212
50%,0.000322,0.000330
75%,0.000398,0.000446
max,0.001342,0.000645


In [55]:
metrics['TPU_24'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000289,0.000341
std,0.000236,0.000357
min,0.000032,0.000097
25%,0.000125,0.000165
50%,0.000260,0.000236
75%,0.000358,0.000293
max,0.000726,0.001314


In [56]:
metrics['TPU_25'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000248,0.000324
std,0.000158,0.000123
min,0.000072,0.000160
25%,0.000095,0.000243
50%,0.000256,0.000297
75%,0.000351,0.000399
max,0.000522,0.000567


In [57]:
metrics['TPU_26'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.002264,0.000358
std,0.005954,0.000138
min,0.000031,0.000141
25%,0.000080,0.000263
50%,0.000268,0.000363
75%,0.000653,0.000420
max,0.019166,0.000588


In [58]:
metrics['TPU_27'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000475,0.000303
std,0.000449,0.000129
min,0.000064,0.000125
25%,0.000154,0.000224
50%,0.000311,0.000270
75%,0.000808,0.000407
max,0.001328,0.000500


In [59]:
metrics['TPU_30'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000206,0.000253
std,0.000147,0.000043
min,0.000045,0.000173
25%,0.000107,0.000239
50%,0.000125,0.000268
75%,0.000334,0.000281
max,0.000436,0.000299


In [60]:
metrics['TPU_31'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000263,0.000363
std,0.000134,0.000107
min,0.000086,0.000197
25%,0.000175,0.000317
50%,0.000251,0.000361
75%,0.000310,0.000458
max,0.000552,0.000497


In [61]:
metrics['TPU_32'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000349,0.000457
std,0.000245,0.000088
min,0.000059,0.000344
25%,0.000223,0.000406
50%,0.000310,0.000442
75%,0.000372,0.000457
max,0.000865,0.000640


In [62]:
metrics['TPU_33'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000410,0.000382
std,0.000377,0.000068
min,0.000081,0.000235
25%,0.000169,0.000352
50%,0.000296,0.000384
75%,0.000506,0.000419
max,0.001344,0.000486


In [63]:
metrics['TPU_34'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000390,0.000274
std,0.000343,0.000142
min,0.000052,0.000159
25%,0.000114,0.000216
50%,0.000221,0.000223
75%,0.000651,0.000281
max,0.000954,0.000662


In [64]:
metrics['TPU_35'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.036321,0.017536
std,0.111247,0.051963
min,0.000356,0.000785
25%,0.000814,0.001002
50%,0.001157,0.001075
75%,0.001811,0.001153
max,0.352932,0.165423


In [65]:
metrics['TPU_36'].describe()

,train_value,eval_value
count,9.000000,9.000000
mean,0.000388,0.000320
std,0.000254,0.000123
min,0.000062,0.000174
25%,0.000269,0.000202
50%,0.000329,0.000327
75%,0.000570,0.000367
max,0.000786,0.000538


In [66]:
metrics['TPU_37'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.024284,0.018984
std,0.074991,0.058688
min,0.000090,0.000200
25%,0.000317,0.000307
50%,0.000369,0.000439
75%,0.000667,0.000619
max,0.237707,0.186013


In [67]:
metrics['TPU_38'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000339,0.000289
std,0.000429,0.000174
min,0.000033,0.000124
25%,0.000106,0.000187
50%,0.000252,0.000259
75%,0.000333,0.000288
max,0.001510,0.000735


In [68]:
metrics['TPU_39'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000999,0.000447
std,0.001924,0.000474
min,0.000056,0.000200
25%,0.000106,0.000226
50%,0.000191,0.000289
75%,0.000687,0.000406
max,0.006269,0.001770


In [69]:
metrics['TPU_40'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000252,0.000253
std,0.000244,0.000128
min,0.000058,0.000097
25%,0.000131,0.000145
50%,0.000178,0.000243
75%,0.000255,0.000319
max,0.000889,0.000460


In [70]:
metrics['TPU_41'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.001860,0.002080
std,0.004652,0.005592
min,0.000036,0.000207
25%,0.000084,0.000220
50%,0.000306,0.000287
75%,0.000883,0.000370
max,0.015050,0.017991


In [71]:
metrics['TPU_42'].describe()

,train_value,eval_value
count,10.000000,10.000000
mean,0.000184,0.000246
std,0.000107,0.000119
min,0.000018,0.000066
25%,0.000126,0.000165
50%,0.000166,0.000250
75%,0.000254,0.000334
max,0.000387,0.000433


In [72]:
metrics['TPU_50'].describe()

,train_value,eval_value
count,30.000000,30.000000
mean,0.000273,0.000268
std,0.000227,0.000126
min,0.000045,0.000078
25%,0.000121,0.000166
50%,0.000204,0.000250
75%,0.000346,0.000331
max,0.000972,0.000573


In [73]:
metrics['TPU_51'].describe()

,train_value,eval_value
count,30.000000,30.000000
mean,0.000268,0.000260
std,0.000184,0.000051
min,0.000032,0.000179
25%,0.000149,0.000224
50%,0.000220,0.000258
75%,0.000409,0.000298
max,0.000771,0.000358


In [74]:
metrics['TPU_52'].describe()

,train_value,eval_value
count,20.000000,20.000000
mean,0.000154,0.000096
std,0.000147,0.000027
min,0.000016,0.000045
25%,0.000052,0.000079
50%,0.000119,0.000094
75%,0.000196,0.000111
max,0.000628,0.000152


In [75]:
import numpy as np
from bokeh.plotting import figure, output_file, show, output_notebook
output_notebook()

Loading BokehJS ...

In [76]:
exps = [key for key in metrics.keys()]

In [77]:
q1 = [metrics[exp]['eval_value'].quantile(q=0.25) for exp in exps]
q2 = [metrics[exp]['eval_value'].quantile(q=0.5) for exp in exps]
q3 = [metrics[exp]['eval_value'].quantile(q=0.75) for exp in exps]
qmin = [metrics[exp]['eval_value'].quantile(q=0.) for exp in exps]
qmax = [metrics[exp]['eval_value'].quantile(q=1.) for exp in exps]

In [78]:
iqr = np.array(q3) - np.array(q1)
upper = np.array(q3) + 1.5*iqr
lower = np.array(q1) - 1.5*iqr

In [79]:
p = figure(title='Performance of Cloud TPU Models for CPE04015',
           tools="",
           width=2400,
           height=600,
           background_fill_color="#efefef",
           x_range=exps,
           toolbar_location=None)

# axis labels
p.xaxis.axis_label = 'Experiment'
p.yaxis.axis_label = 'Evaluation Loss Value'

# stems
p.segment(exps, upper, exps, q3, line_color="black")
p.segment(exps, lower, exps, q1, line_color="black")

# boxes
p.vbar(exps, 0.1, q2, q3, fill_color="#E08E79", line_color="black")
p.vbar(exps, 0.1, q1, q2, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(exps, lower, 0.1, 0.000002, line_color="black")
p.rect(exps, upper, 0.1, 0.000002, line_color="black")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="10pt"

output_file("boxplot.html", title='Cloud TPU Training')

show(p)

In [80]:
# create an empty dataframe for a summary of experiment results
columns = ['id', 'm', 'hourly', 'daily', 'weekly', 'use_timestamps', 'dense',
           'train_specs', 'eval_loss_avg', 'eval_loss_std', 'no_exp']

summary = pd.DataFrame(columns = columns)

In [81]:
for experiment in metrics.keys():
    df = metrics[experiment][-1:][['id', 'm', 'hourly', 'daily', 'weekly',
                                  'use_timestamps', 'dense', 'train_specs']]
    df['eval_loss_avg'] = metrics[experiment]['eval_value'].mean()
    df['eval_loss_std'] = metrics[experiment]['eval_value'].std()
    df['no_exp'] = metrics[experiment][-1:]['exp'] + 1
    summary = pd.concat([summary, df], ignore_index=True)

In [82]:
# ToDo: retrieve the average execution time for each experiment!!!
summary.sort_values('eval_loss_avg')

,id,m,hourly,daily,weekly,use_timestamps,dense,train_specs,eval_loss_avg,eval_loss_std,no_exp
32,TPU_52,"[8, 8, 4]","[64, 128]","[64, 128]","[16, 64]",True,"[512, 128, 8]","[0.001, 31000, 20000, 32]",0.000096,0.000027,20
29,TPU_42,"[8, 8, 4]","[64, 128]","[64, 128]","[16, 64]",True,"[512, 128, 8]","[0.005, 16000, 20000, 32]",0.000246,0.000119,10
27,TPU_40,"[8, 8, 4]","[64, 128]","[64, 128]","[64, 128]",True,"[512, 128, 8]","[0.008, 16000, 20000, 32]",0.000253,0.000128,10
17,TPU_30,"[8, 8, 4]","[64, 128]","[64, 128]","[16, 64]",True,"[512, 128, 8]","[0.008, 16000, 20000, 32]",0.000253,0.000043,10
31,TPU_51,"[8, 8, 4]","[64, 128]","[64, 128]","[16, 64]",True,"[512, 128, 8]","[0.001, 16000, 20000, 32]",0.000260,0.000051,30
30,TPU_50,"[8, 8, 4]","[64, 128]","[64, 128]","[16, 64]",True,"[512, 128, 8]","[0.005, 16000, 20000, 32]",0.000268,0.000126,30
21,TPU_34,"[8, 8, 4]","[64, 256]","[64, 256]","[16, 128]",True,"[1024, 256, 16]","[0.008, 16000, 20000, 32]",0.000274,0.000142,10
25,TPU_38,"[8, 8, 4]","[64, 128, 256]","[64, 128, 256]","[32, 64, 128]",True,"[512, 128, 8]","[0.008, 16000, 20000, 32]",0.000289,0.000174,10
9,TPU_20,"[8, 8, 4]","[64, 64, 64, 64]","[64, 64, 64, 64]","[32, 32]",True,"[256, 64, 8]","[0.008, 16000, 20000, 32]",0.000293,0.000155,10
16,TPU_27,"[8, 8, 4]","[128, 128, 128, 128]","[128, 128, 128, 128]","[64, 64]",True,"[512, 128, 8]","[0.008, 16000, 20000, 32]",0.000303,0.000129,10
